In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
# from torchvision.datasets.utils import download_url
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader
from GANs_model import GANs_Discriminator, GANs_Generator
from model_trainer import train_discriminator, train_generator
from model_fit import SG_fit, G_fit
from test_model import test_model, test_model_lists
from Abalone_data_preprocessing import two_classes_Abalone, four_classes_Abalone, get_features, get_labels, GANs_two_class_real_data, GANs_four_class_real_data
from choose_device import get_default_device, to_device, DeviceDataLoader
from fit import f1
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
from sklearn.ensemble import RandomForestClassifier


In [2]:

#print("This is train.py file")


def shuffle_in_unison(a, b):     #Shuffling the features and labels in unison.
    assert len(a) == len(b)       #In Python, the assert statement is used to continue the execute if the given condition evaluates to True.
    shuffled_a = np.empty(a.shape, dtype=a.dtype)       #Return a new array of given shape and type, without initializing entries.
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b


def Oversampled_data(X_train_SMOTE, y_train_SMOTE, y_train, device):
    X_oversampled_0 = []
    X_oversampled_2 = []
    X_oversampled_3 = []
    for i in y_train_SMOTE[(y_train.shape[0]):]:
        if i==0: 
            X_oversampled_0.append(X_train_SMOTE[i])
        elif i==2: 
            X_oversampled_2.append(X_train_SMOTE[i])
        elif i==3: 
            X_oversampled_3.append(X_train_SMOTE[i])
    X_oversampled_0 = torch.from_numpy(np.array(X_oversampled_0))
    X_oversampled_0 = to_device(X_oversampled_0.float(), device)
    X_oversampled_2 = torch.from_numpy(np.array(X_oversampled_2))
    X_oversampled_2 = to_device(X_oversampled_2.float(), device)
    X_oversampled_3 = torch.from_numpy(np.array(X_oversampled_3))
    X_oversampled_3 = to_device(X_oversampled_3.float(), device)

    return X_oversampled_0, X_oversampled_2, X_oversampled_3


In [3]:
def model_rf(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(n_estimators=10, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1_mes = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    return accuracy, f1_mes, precision, recall, model

In [4]:

Abalone_df  = pd.read_csv('../hfcr.csv')
# Abalone_df.dropna(inplace=True)
minority = 1
majority = 0


In [5]:
device = torch.device('cuda')

X_train, X_test, y_train, y_test = train_test_split(Abalone_df.drop(['Class'], axis=1), Abalone_df['Class'], test_size=0.2, random_state=10)

#### Calculating train and test accuracy and f1 score of non oversampled training data ####
Normal_accuracy, Normal_f1_score, Normal_precision, Normal_recall, model_normal = model_rf(X_train, y_train, X_test, y_test) 
print("Normal_accuracy: ", Normal_accuracy)
print("Normal_f1_score: ", Normal_f1_score)
print("Normal_precision: ", Normal_precision)
print("Normal_recall: ", Normal_recall)
print(classification_report(y_test, model_normal.predict(X_test)))



Normal_accuracy:  0.8166666666666667
Normal_f1_score:  0.8182072829131654
Normal_precision:  0.8202380952380951
Normal_recall:  0.8166666666666667
              precision    recall  f1-score   support

           0       0.88      0.86      0.87        43
           1       0.67      0.71      0.69        17

    accuracy                           0.82        60
   macro avg       0.77      0.78      0.78        60
weighted avg       0.82      0.82      0.82        60



In [6]:

print("Before OverSampling, counts of label '0': {}".format(sum(y_train==0)))
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1))) 
# print("Before OverSampling, counts of label '2': {}".format(sum(y_train==2)))
# print("Before OverSampling, counts of label '3': {}".format(sum(y_train==3)))     


Before OverSampling, counts of label '0': 160
Before OverSampling, counts of label '1': 79


In [7]:
X_train_SMOTE,y_train_SMOTE = SMOTE().fit_resample(X_train,y_train)

print('After OverSampling, the shape of train_X: {}'.format(X_train_SMOTE.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_SMOTE.shape))

print("After OverSampling, counts of label '0': {}".format(sum(y_train_SMOTE==0)))
print("After OverSampling, counts of label '1': {}".format(sum(y_train_SMOTE==1))) 


After OverSampling, the shape of train_X: (320, 12)
After OverSampling, the shape of train_y: (320,) 

After OverSampling, counts of label '0': 160
After OverSampling, counts of label '1': 160


In [8]:
device = torch.device('cuda')

#### Calculating train and test accuracy and f1 score of SMOTE oversampled training data ####

Smote_accuracy, Smote_f1_score, Smote_precision, Smote_recall, model_smote = model_rf(X_train_SMOTE, y_train_SMOTE, X_test, y_test) 
print("Smote_accuracy: ", Smote_accuracy)
print("Smote_f1_score: ", Smote_f1_score)
print("Smote_precision: ", Smote_precision)
print("Smote_recall: ", Smote_recall)
print(classification_report(y_test, model_smote.predict(X_test)))


Smote_accuracy:  0.8333333333333334
Smote_f1_score:  0.8333333333333334
Smote_precision:  0.8333333333333334
Smote_recall:  0.8333333333333334
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        43
           1       0.71      0.71      0.71        17

    accuracy                           0.83        60
   macro avg       0.79      0.79      0.79        60
weighted avg       0.83      0.83      0.83        60



In [9]:
torch.cuda.is_available()

True

In [10]:
##################### TWO CLASS ABALONE #####################
##### Oversampled data from SMOTE that is now to be passed in SMOTified GANs #####
X_oversampled = X_train_SMOTE[(X_train.shape[0]):].to_numpy()
X_oversampled = torch.from_numpy(X_oversampled)
X_oversampled = to_device(X_oversampled.float(), device)


In [11]:
#print(X_oversampled.shape)
lr = 0.001
epochs = 150
batch_size = 128

X_real, y_real = GANs_two_class_real_data(X_train, y_train)   #Defining the real data to be put in GANs



<class 'pandas.core.frame.DataFrame'>


c:\Users\Vinu's PC\Documents\GANclassimbalanced\SMOTified_GANs_code\Abalone_data_preprocessing.py:64: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()


In [12]:
#Training our SMOTified GANs and GANs model and fetching their trained generators.
generator_SG, generator_G = f1(X_train, y_train, X_train_SMOTE, y_train_SMOTE, X_real, y_real, X_oversampled, device, lr, epochs, batch_size, minority, majority)

Trained_X_oversampled_SG = generator_SG(X_oversampled.float().to(device)).cpu().detach().numpy()
Trained_SG_dataset = np.concatenate((X_train_SMOTE[:(X_train.shape[0])], Trained_X_oversampled_SG), axis=0)
X_trained_SG, y_trained_SG = shuffle_in_unison(Trained_SG_dataset, y_train_SMOTE)



  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1/150], loss_g: 0.8521, loss_d: 0.3271, real_score: -1736.0905, fake_score: 0.0796


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [2/150], loss_g: 0.7909, loss_d: 4730.9805, real_score: 9461.3848, fake_score: 0.2491


  0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch [148/150], loss_g: 1.1847, loss_d: 0.1837, real_score: -34328.0312, fake_score: 0.8119


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [149/150], loss_g: 1.1902, loss_d: 0.1825, real_score: -34322.3789, fake_score: 0.8197


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [150/150], loss_g: 1.1956, loss_d: 0.1813, real_score: -34316.7266, fake_score: 0.8275


In [13]:
#### Calculating train and test accuracy and f1 score of SMOTified GANs oversampled training data ####
# SG_test_accuracy, SG_train_accuracy, SG_f1_score = model_rf(X_trained_SG, y_trained_SG, X_test, y_test)


SG_accuracy, SG_f1_score, SG_precision, SG_recall, model_SG = model_rf(X_trained_SG, y_trained_SG, X_test, y_test)
print("SG_accuracy: ", SG_accuracy)
print("SG_f1_score: ", SG_f1_score)
print("SG_precision: ", SG_precision)
print("SG_recall: ", SG_recall)
print(classification_report(y_test, model_SG.predict(X_test)))



SG_accuracy:  0.8
SG_f1_score:  0.7911111111111111
SG_precision:  0.7908346972176759
SG_recall:  0.8
              precision    recall  f1-score   support

           0       0.83      0.91      0.87        43
           1       0.69      0.53      0.60        17

    accuracy                           0.80        60
   macro avg       0.76      0.72      0.73        60
weighted avg       0.79      0.80      0.79        60



c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [14]:

GANs_noise = torch.randn((X_oversampled.shape[0]), (X_oversampled.shape[1]), device=device)
Trained_X_oversampled_G = generator_G(GANs_noise.float().to(device)).cpu().detach().numpy()
Trained_G_dataset = np.concatenate((X_train_SMOTE[:(X_train.shape[0])], Trained_X_oversampled_G), axis=0)
X_trained_G, y_trained_G = shuffle_in_unison(Trained_G_dataset, y_train_SMOTE)

In [15]:
#### Calculating train and test accuracy and f1 score of SMOTified GANs oversampled training data ####
# G_test_accuracy, G_train_accuracy, G_f1_score = test_model_lists(X_trained_G, y_trained_G.ravel(), X_test, y_test.ravel(), 30)



G_accuracy, G_f1_score, G_precision, G_recall, model_G = model_rf(X_trained_G, y_trained_G, X_test, y_test)
print("G_accuracy: ", G_accuracy)
print("G_f1_score: ", G_f1_score)
print("G_precision: ", G_precision)
print("G_recall: ", G_recall)
print(classification_report(y_test, model_G.predict(X_test)))


G_accuracy:  0.85
G_f1_score:  0.8485893416927899
G_precision:  0.8477272727272727
G_recall:  0.85
              precision    recall  f1-score   support

           0       0.89      0.91      0.90        43
           1       0.75      0.71      0.73        17

    accuracy                           0.85        60
   macro avg       0.82      0.81      0.81        60
weighted avg       0.85      0.85      0.85        60



c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [16]:

print("Normal Accuracy: ",round(Normal_accuracy,3))
print("SMOTE Accuracy: ",round(Smote_accuracy,3))
print("G Accuracy: ",round(G_accuracy,3))
print("SG Accuracy: ",round(SG_accuracy,3))



print("Normal F1 Score: ",round(Normal_f1_score,3))
print("Normal Precision: ",round(Normal_precision,3))
print("Normal Recall: ",round(Normal_recall,3))

print("SMOTE F1 Score: ",round(Smote_f1_score,3))
print("SMOTE Precision: ",round(Smote_precision,3))
print("SMOTE Recall: ",round(Smote_recall,3))

print("G F1 Score: ",round(G_f1_score,3))
print("G Precision: ",round(G_precision,3))
print("G Recall: ",round(G_recall,3))

print("SG F1 Score: ",round(SG_f1_score,3))
print("SG Precision: ",round(SG_precision,3))
print("SG Recall: ",round(SG_recall,3))



Normal Accuracy:  0.817
SMOTE Accuracy:  0.833
G Accuracy:  0.85
SG Accuracy:  0.8
Normal F1 Score:  0.818
Normal Precision:  0.82
Normal Recall:  0.817
SMOTE F1 Score:  0.833
SMOTE Precision:  0.833
SMOTE Recall:  0.833
G F1 Score:  0.849
G Precision:  0.848
G Recall:  0.85
SG F1 Score:  0.791
SG Precision:  0.791
SG Recall:  0.8


In [17]:
print("NORMAL MODEL")
print(classification_report(y_test, model_normal.predict(X_test)))
print("SMOTE MODEL")
print(classification_report(y_test, model_smote.predict(X_test)))
print("GAN MODEL")
print(classification_report(y_test, model_G.predict(X_test)))
print("SMOTE GAN MODEL")
print(classification_report(y_test, model_SG.predict(X_test)))



NORMAL MODEL
              precision    recall  f1-score   support

           0       0.88      0.86      0.87        43
           1       0.67      0.71      0.69        17

    accuracy                           0.82        60
   macro avg       0.77      0.78      0.78        60
weighted avg       0.82      0.82      0.82        60

SMOTE MODEL
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        43
           1       0.71      0.71      0.71        17

    accuracy                           0.83        60
   macro avg       0.79      0.79      0.79        60
weighted avg       0.83      0.83      0.83        60

GAN MODEL
              precision    recall  f1-score   support

           0       0.89      0.91      0.90        43
           1       0.75      0.71      0.73        17

    accuracy                           0.85        60
   macro avg       0.82      0.81      0.81        60
weighted avg       0.85      0.85      

c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Vinu's PC\.conda\envs\py310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
